# Pré traitement des données

## I. Conversion C3D en JSON rééchantilloné à 50 Hz

In [1]:
import numpy as np
import ezc3d
import json
from scipy.signal import resample,butter, lfilter, freqz

In [7]:
def recuperer_evenements(file_path):
    """
    Cette fonction extrait les événements du fichier C3D.

    Args:
        file_path (str): Le chemin du fichier C3D.

    Returns:
        dict: Un dictionnaire contenant les événements et leur temps correspondant.
    """
    c3d = ezc3d.c3d(file_path)
    events = c3d['parameters']['EVENT']['LABELS']['value'] # Récupérer les événements
    frames = c3d['parameters']['EVENT']['TIMES']['value'] # Récupérer les temps correspondant aux événements
    events_dict = {}
    for event, frame in zip(events, frames): # Associer les événements à leur temps correspondant
        events_dict[event] = frame 
    return events_dict

def butter_lowpass(cutoff, fs, order=5):
    """
    Crée les coefficients d'un filtre Butterworth passe-bas.

    Args:
        cutoff (float): Fréquence de coupure du filtre.
        fs (float): Fréquence d'échantillonnage du signal.
        order (int): Ordre du filtre.

    Returns:
        tuple: Coefficients du filtre (b, a).
    """
    nyq = 0.5 * fs # Fréquence de Nyquist
    normal_cutoff = cutoff / nyq # Fréquence de coupure normalisée
    b, a = butter(order, normal_cutoff, btype='low', analog=False) # Création des coefficients du filtre
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    """
    Applique un filtre Butterworth passe-bas à un signal.

    Args:
        data (array_like): Signal à filtrer.
        cutoff (float): Fréquence de coupure du filtre.
        fs (float): Fréquence d'échantillonnage du signal.
        order (int): Ordre du filtre.

    Returns:
        array_like: Signal filtré.
    """
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def reechantillonnage_fc_coupure_et_association_labels_et_data(file_path, cutoff_freq=20, target_freq=50):
    """
    Cette fonction transforme les données échantillonnées à 2100 Hz en données échantillonnées à 50 Hz,
    puis applique un filtre Butterworth passe-bas avec une fréquence de coupure de 20 Hz, et enfin associe les étiquettes des données aux données rééchantillonnées et filtrées.

    Args:
        file_path (str): Le chemin du fichier C3D.
        cutoff_freq (float): Fréquence de coupure du filtre Butterworth (20 Hz).
        target_freq (float): Fréquence cible après rééchantillonnage (50 Hz).

    Returns:
        tuple: Un tuple contenant un dictionnaire fusionnant les étiquettes et les données rééchantillonnées et filtrées, et les temps rééchantillonnés.
    """
    c3d = ezc3d.c3d(file_path)
    labels = c3d['parameters']['ANALOG']['LABELS']['value'] # Récupérer les étiquettes des données
    original_freq = c3d['parameters']['ANALOG']['RATE']['value'][0] # Récupérer la fréquence d'échantillonnage
    data = c3d['data']['analogs'] # Récupérer les données
    nb_frame = len(data[0][0]) # Récupérer le nombre de frames
    target_freq = 50  # fréquence de rééchantillonnage
    
    # Rééchantillonnage des données
    nb_samples_target = int(nb_frame * (target_freq / original_freq)) # Calcul du nombre d'échantillons cible
    resampled_times = np.linspace(0., (nb_frame / original_freq), num=nb_samples_target) # Création d'un tableau de temps rééchantillonné
    resampled_data = np.zeros((len(labels), nb_samples_target)) # Création d'un tableau de zéros de la taille des données rééchantillonnées
    for i in range(len(labels)): # Pour chaque étiquette
        resampled_data[i, :] = resample(data[0][i, :], nb_samples_target) # Rééchantillonnage des données

    fusion_label_data = {}
    for i, label in enumerate(labels): # Pour chaque étiquette
        fusion_label_data[label] = resampled_data[i, :] # Associer les étiquettes aux données rééchantillonnées
        
    # Filtrage des données rééchantillonnées
    filtered_data = np.zeros_like(resampled_data) # Création d'un tableau de zéros de la même taille que les données rééchantillonnées
    for i in range(len(labels)): # Pour chaque étiquette
        filtered_data[i, :] = butter_lowpass_filter(resampled_data[i, :], cutoff_freq, target_freq) # Appliquer un filtre passe-bas

    fusion_label_data = {}
    for i, label in enumerate(labels): 
        fusion_label_data[label] = filtered_data[i, :] # Associer les étiquettes aux données filtrées
    
    return fusion_label_data, resampled_times
    

def filtrer_labels(fusion_label_data):
    """
    Cette fonction filtre les étiquettes des données pour ne garder que celles liées à l'accéléromètre et au gyroscope.

    Args:
        fusion_label_data (dict): Dictionnaire contenant les étiquettes et les données associées.

    Returns:
        tuple: Un tuple contenant une liste des étiquettes filtrées et un dictionnaire des données associées aux étiquettes filtrées.
    """
    labels_filtre = []
    labels_data_filtre = {}
    for label, valeurs in fusion_label_data.items(): # Pour chaque étiquette et ses valeurs
        if 'ACC' in label or 'GYRO' in label: # Si l'étiquette contient 'ACC' ou 'GYRO'
            labels_filtre.append(label) # Ajouter l'étiquette à la liste des étiquettes filtrées
            labels_data_filtre[label] = valeurs # Ajouter les valeurs associées à l'étiquette dans le dictionnaire des données filtrées
    return labels_filtre, labels_data_filtre

def calcul_norme(labels_data_filtre):
    """
    Cette fonction calcule les normes des données filtrées.

    Args:
        labels_data_filtre (dict): Dictionnaire contenant les étiquettes et les données associées, filtrées.

    Returns:
        dict: Un dictionnaire contenant les normes calculées.
    """
    normes = {}
    traite = set() # Créer un ensemble vide pour stocker les capteurs, les côtés et les mesures déjà traités
    for key, value in labels_data_filtre.items():
        parts = key.split('_') # Séparer l'étiquette en parties
        sensor = parts[0] # Récupérer le capteur
        side = parts[1] # Récupérer le côté
        measure = parts[2] # Récupérer la mesure (GYRO ou ACC)
        
        if (sensor, side, measure) not in traite: # Si le capteur, le côté et la mesure n'ont pas déjà été traités
            traite.add((sensor, side, measure)) # Ajouter le capteur, le côté et la mesure à l'ensemble des éléments traités
            
            if "ACC" in measure:
                # Obtention des axes X, Y et Z
                axe_X = labels_data_filtre[f'{sensor}_{side}_{measure}_X']
                axe_Y = labels_data_filtre[f'{sensor}_{side}_{measure}_Y']
                axe_Z = labels_data_filtre[f'{sensor}_{side}_{measure}_Z']
            
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2) - 1 # Calcul de la norme auquelle on soustrait 1 pour enlever la gravité
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme
                
            else:
                axe_X = labels_data_filtre[f'{sensor}_{side}_{measure}_X']
                axe_Y = labels_data_filtre[f'{sensor}_{side}_{measure}_Y']
                axe_Z = labels_data_filtre[f'{sensor}_{side}_{measure}_Z']
        
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2) 
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme

    return normes

def creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, resampled_times, events_dict, normes):
    """
    Cette fonction crée une structure JSON à partir des données filtrées et d'autres informations.

    Args:
        labels_data_filtre (dict): Dictionnaire contenant les étiquettes et les données associées, filtrées.
        patient_id (int): Identifiant du patient.
        date_de_naissance (str): Date de naissance du patient.
        medicaments (str): Médicaments pris par le patient.
        resampled_times (ndarray): Temps rééchantillonné.
        events_dict (dict): Dictionnaire contenant les événements et leur temps correspondant.
        normes (dict): Dictionnaire contenant les normes calculées.

    Returns:
        dict: Structure JSON contenant les données et les métadonnées.
    """
    json_data = {
        "metadata": {
            "Détails du patient": {
                "Identifiant": patient_id,
                "Date de naissance": date_de_naissance,
                "Medicaments": medicaments
            },
            "Temps": resampled_times.tolist()
        }
    }
    
    for key, value in labels_data_filtre.items():
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]
        
        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        json_data[sensor][side][measure][axis] = value.tolist()
        
    for key in normes:
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]

        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        # Insérer la norme au même niveau d'indentation que l'axe
        json_data[sensor][side][measure][axis] = value.tolist()
        json_data[sensor][side][measure]["norme"] = normes[key].tolist()
    
    # Ajouter les événements FOG
    json_data["FOG"] = {
        "Debut": events_dict["FOG_begin"].tolist(),
        "Fin": events_dict["FOG_end"].tolist()
    }
    
    return json_data

def creation_json_grace_c3d(file_path, patient_id, date_de_naissance, medicaments, output_path):
    """
    Cette fonction crée un fichier JSON à partir d'un fichier C3D.

    Args:
        file_path (str): Le chemin du fichier C3D.
        patient_id (int): Identifiant du patient.
        date_de_naissance (str): Date de naissance du patient.
        medicaments (str): Médicaments pris par le patient.
        output_path (str): Le chemin de sortie du fichier JSON.
    """
    events_dict = recuperer_evenements(file_path) 
    fusion_label_data, resampled_times = reechantillonnage_fc_coupure_et_association_labels_et_data(file_path)
    labels_filtre, labels_data_filtre = filtrer_labels(fusion_label_data)
    normes = calcul_norme(labels_data_filtre)
    json_structure = creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, resampled_times, events_dict, normes)
    
    with open(output_path, "w") as fichier_json:
        json.dump(json_structure, fichier_json, indent=4)

# Utilisation de la fonction pour traiter le fichier C3D et générer le fichier JSON
fichier_brut = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video_overlay_3.c3d"
patient_id = 1234
date_de_naissance = 45
medicaments = "ON"
chemin_sortie = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_3.json"

creation_json_grace_c3d(fichier_brut, patient_id, date_de_naissance, medicaments, chemin_sortie)


In [37]:
# Charger le fichier JSON
chemin_fichier_json = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_3.json"
with open(chemin_fichier_json, "r") as fichier_json:
    donnees_patient = json.load(fichier_json)

# Accéder aux données spécifiques
#temps = donnees_patient["Rectus Femoris"]["Left"]["ACC"]["norme"]
temps = donnees_patient["metadata"]["Temps"]
gyro_x= donnees_patient["Rectus Femoris"]["Left"]["GYRO"]["X"]

## Conversion C3D en JSON sans rééchantillonnage

In [2]:
def recuperer_evenements(file_path):
    c3d = ezc3d.c3d(file_path)
    events = c3d['parameters']['EVENT']['LABELS']['value']
    frames = c3d['parameters']['EVENT']['TIMES']['value']
    events_dict = {}
    for event, frame in zip(events, frames):
        events_dict[event] = frame
    return events_dict

def associer_labels_et_data(file_path):
    c3d = ezc3d.c3d(file_path)
    labels = c3d['parameters']['ANALOG']['LABELS']['value']
    frequence = c3d['parameters']['ANALOG']['RATE']['value'][0]
    data = c3d['data']['analogs']
    nb_frame = len(data[0][0])
    temps = np.linspace(0., nb_frame / frequence, num=nb_frame)
    fusion_label_data = {}
    for i, label in enumerate(labels):
        fusion_label_data[label] = data[0][i, :]
    return fusion_label_data, temps   # Dictionnaire avec tous les labels associés aux données

def filtrer_labels(fusion_label_data):
    labels_filtre = []
    labels_data_filtre = {}
    for label, valeurs in fusion_label_data.items():
        if 'ACC' in label or 'GYRO' in label:
            labels_filtre.append(label)
            labels_data_filtre[label] = valeurs
    return labels_filtre, labels_data_filtre # Dictionnaire avec les labels filtrés et les données associées

def calcul_norme(labels_data_filtre):
    normes = {}
    traite = set()
    for key, value in labels_data_filtre.items():
        parts = key.split('_')
        sensor = parts[0]
        side = parts[1]
        measure = parts[2]
        axes = parts[3]
        
        if (sensor, side, measure) not in traite:
            traite.add((sensor, side, measure))
            
            if "ACC" in measure:
                # Obtention des axes X, Y et Z
                axe_X = labels_data_filtre[f'{sensor}_{side}_{measure}_X']
                axe_Y = labels_data_filtre[f'{sensor}_{side}_{measure}_Y']
                axe_Z = labels_data_filtre[f'{sensor}_{side}_{measure}_Z']
            
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2) - 1
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme
                
            else :
                axe_X = labels_data_filtre[f'{sensor}_{side}_{measure}_X']
                axe_Y = labels_data_filtre[f'{sensor}_{side}_{measure}_Y']
                axe_Z = labels_data_filtre[f'{sensor}_{side}_{measure}_Z']
        
                norme = np.sqrt(axe_X**2 + axe_Y**2 + axe_Z**2)
                nom_cle = f'{sensor}_{side}_{measure}_norme'
                normes[nom_cle] = norme

    return normes

def creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, temps, events_dict, normes):
    json_data = {
        "metadata": {
            "Détails du patient": {
                "Identifiant": patient_id,
                "Date de naissance": date_de_naissance,
                "Medicaments": medicaments
            },
            "Temps": temps.tolist()
        }
    }
    
    for key, value in labels_data_filtre.items():
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]
        
        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        json_data[sensor][side][measure][axis] = value.tolist()
        
    for key in normes:
        parts = key.split('_')
        sensor = parts[1]
        side = parts[0]
        measure = parts[2]
        axis = parts[3]

        if sensor not in json_data:
            json_data[sensor] = {}

        if side not in json_data[sensor]:
            json_data[sensor][side] = {}

        if measure not in json_data[sensor][side]:
            json_data[sensor][side][measure] = {}

        # Insérer la norme au même niveau d'indentation que l'axe
        json_data[sensor][side][measure][axis] = value.tolist()
        json_data[sensor][side][measure]["norme"] = normes[key].tolist()
    
        # Ajouter les événements FOG
    json_data["FOG"] = {
        "Debut": events_dict["FOG_begin"].tolist(),
        "Fin": events_dict["FOG_end"].tolist()
    }
    
    
    return json_data

    


def creation_json_grace_c3d(file_path, patient_id, date_de_naissance, medicaments, output_path):
    events_dict = recuperer_evenements(file_path) 
    fusion_label_data, temps = associer_labels_et_data(file_path)
    labels_filtre, labels_data_filtre = filtrer_labels(fusion_label_data)
    normes = calcul_norme(labels_data_filtre)
    json_structure = creer_structure_json(labels_data_filtre, patient_id, date_de_naissance, medicaments, temps, events_dict, normes)
    
    with open(output_path, "w") as fichier_json:
        json.dump(json_structure, fichier_json, indent=4)

# Utilisation de la fonction pour traiter le fichier C3D et générer le fichier JSON
fichier_brut = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video_overlay_3.c3d"
patient_id = 1234
date_de_naissance = 45
medicaments = "ON"
chemin_sortie = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_3.json"

creation_json_grace_c3d(fichier_brut, patient_id, date_de_naissance, medicaments, chemin_sortie)


In [3]:
# Charger le fichier JSON
chemin_fichier_json = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATABASE/video_overlay_3.json"
with open(chemin_fichier_json, "r") as fichier_json:
    donnees_patient = json.load(fichier_json)

# Accéder aux données spécifiques
accelerometre_left_Z_rectus_femoris = donnees_patient["Vastus Lateralis"]["Left"]["GYRO"]["norme"]